# 21기 KNN 정규세션 과제

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [7]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.neighbors import KNeighborsRegressor
import numpy as np


In [3]:
df.isna().sum()

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64

In [8]:
df.drop(['Product_Category_3'], axis=1, inplace=True)
df = df.fillna(df.mean()['Product_Category_2']) ## category_2는 평균값으로 대체
df.drop(['User_ID', 'Product_ID'], axis=1, inplace=True)

<ipython-input-8-2f4922b00a8d>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean()['Product_Category_2']) ## category_2는 평균값으로 대체


In [12]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].map({'0':0, '1':1, '2':2, '3':3, '4+':4})

In [13]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler()
df_std = pd.DataFrame(standard_scaler.fit_transform(df[['Occupation', 'Stay_In_Current_City_Years','Marital_Status', 'Product_Category_1', 'Product_Category_2']]), columns=['Occupation', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2'])
df_std[['Age', 'Gender', 'City_Category']] = df[['Age', 'Gender', 'City_Category']]
df_std = pd.get_dummies(df_std, columns=['Age', 'Gender', 'City_Category'])

In [ ]:
df_std.info()

## 2. KNN 구현 & 파라미터 튜닝

In [15]:
from sklearn.model_selection import train_test_split

x = df_std
y = df['Purchase']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=516)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3498, 17) (1500, 17) (3498,) (1500,)


In [23]:
for k in range(1, 10):
    knn_regressor = KNeighborsRegressor(n_neighbors=k)
    knn_regressor.fit(X_train, y_train)
    print('R_squared : ',knn_regressor.score(X_train, y_train))

R_squared :  0.9388688276295567
R_squared :  0.6641907159681141
R_squared :  0.5401161175111318
R_squared :  0.462047031918258
R_squared :  0.41590702648830735
R_squared :  0.38889548208544467
R_squared :  0.3666548522072811
R_squared :  0.34942732570222557
R_squared :  0.3385523100143468


In [24]:
for k in range(1, 10):
    knn_regressor = KNeighborsRegressor(n_neighbors=k)
    knn_regressor.fit(X_train, y_train)
    print('R_squared : ',knn_regressor.score(X_test, y_test))

R_squared :  -0.31186057026639236
R_squared :  -0.0032856077687006824
R_squared :  0.09619884351096908
R_squared :  0.12934662239243788
R_squared :  0.166214199435048
R_squared :  0.1885146759544133
R_squared :  0.20372879697413282
R_squared :  0.20588191644147535
R_squared :  0.20805971880597562


weight : Distance로 설정

In [25]:
for k in range(1, 10):
    knn_regressor = KNeighborsRegressor(n_neighbors=k, weights = 'distance')
    knn_regressor.fit(X_train, y_train)
    print('R_squared : ',knn_regressor.score(X_train, y_train))

R_squared :  0.9388688276295567
R_squared :  0.9671638513773819
R_squared :  0.9684552412433504
R_squared :  0.9687398018873115
R_squared :  0.9687660694397715
R_squared :  0.9687353850552611
R_squared :  0.9687696383811236
R_squared :  0.9687696383811214
R_squared :  0.968769638381119


In [26]:
for k in range(1, 10):
    knn_regressor = KNeighborsRegressor(n_neighbors=k ,weights = 'distance')
    knn_regressor.fit(X_train, y_train)
    print('R_squared : ',knn_regressor.score(X_test, y_test))

R_squared :  -0.31186057026639236
R_squared :  -0.021560749328599194
R_squared :  0.07705850499065758
R_squared :  0.11068925192187618
R_squared :  0.144639837655145
R_squared :  0.16577676172778055
R_squared :  0.18179624145543438
R_squared :  0.1876064003896727
R_squared :  0.18947260044368164


## 3. Evaluation

In [ ]:
'''훈련 데이터에 대해선
weights를 다르게 할 경우 훈련이 잘 되지만
k값을 바꾸거나 가중치방법을 바꿔도
test 스코어의 차이가 크게 개선되지 않는다.'''